In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else 'cpu'
device

In [ ]:
# model = "facebook/hubert-large-ls960-ft" # for all languages
# model = "openai/whisper-base.en" # for english
# model = "ai4bharat/indicwav2vec-hindi" # for hindi, gujarati
# model = "facebook/wav2vec2-large-960h-lv60-self" # for deu, eng, fra, pol

# TODO: implement language adapters load_adapter() function  
# model = "facebook/mms-1b-fl102" # ara, cmn-script_simplified, deu, eng, fra, guj, hin, jpn, pol, tam, tgl, urd-script_arabic, yue-script_traditional



In [ ]:
language = 'hin'
transcriber = pipeline("automatic-speech-recognition", model=model,model_kwargs={"target_lang": language, "ignore_mismatched_sizes": True}, device=device)

In [ ]:
# def transcribe(audio):
#     sr, y = audio
#     y = (y.astype(np.float32))/np.max(np.abs(y))
#     return transcriber({"sampling_rate": sr, "raw": y})["text"]

# demo = gr.Interface(transcribe, gr.Audio(sources=["microphone"]), "text")

# demo.launch()

In [ ]:
def transcribe_with_stream(stream, new_chunk):
    sr, y = new_chunk
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    if stream is not None:
        stream = np.concatenate([stream, y])
    else:
        stream = y

    transcribed = transcriber({"sampling_rate": sr, "raw": stream})
    print(transcribed)
    text = transcribed["text"]
    return stream, text

demo = gr.Interface(
    transcribe_with_stream,
    ["state", gr.Audio(sources=["microphone"], streaming=True)],
    ["state", "text"],
    live=True,
)

demo.launch()
